In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [13]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
gis_server = config['srv']['server']
pip_server = config['srv']['pip']
gis = config['db']['parksgis']
pip = config['db']['pip']

In [14]:
con_string = 'Driver={' + driver + '};Server=' + pip_server +';Database=' + pip + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
pip_engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [42]:
con_string = 'Driver={' + driver + '};Server=' + gis_server +';Database=' + gis + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
gis_engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Original GIS Data

In [23]:
#Define the tables that will be queried and interacted with
gis_tables = ['property_evw', 'playground_evw', 'zone_evw', 'unmapped_gisallsites_evw', 
              'schoolyard_to_playground_evw', 'greenstreet_evw', 'golfcourse_evw', 
              'restrictivedeclarationsite_evw']

In [24]:
#Create the list of SQL Queries
gis_sql_list = ['select * from parksgis.dpr.'+ t for t in gis_tables]

In [25]:
#Create a list of dataframes with the original data
gis_df_list = [pd.read_sql(con = gis_engine, sql = q) for q in gis_sql_list]

## Original PIP Data

In [27]:
#Define the tables that will be queried and interacted with
pip_tables = ['tbl_ref_allsites', 'tbl_ref_allsites_nosync', 'tbl_pip_allsites', 
              'vw_pip_compatible_inspected_sites', 'allsites']

In [28]:
#Create the list of SQL Queries
pip_sql_list = ['select * from accessnewpip.dbo.'+ t for t in pip_tables]

In [29]:
#Create a list of dataframes with the original dat
pip_df_list = [pd.read_sql(con = pip_engine, sql = q) for q in pip_sql_list]

In [30]:
pip_df_list[0]

,PropNum,Prop ID,Boro,AMPSDistrict,Prop Name,Site Name,Prop Location,Site Location,jurisdiction,typecategory,acres,gisobjid,sourcefc,created_date,row_hash,gis_deleted
0,B001,B001,B,01,American Playground,American Playground,Franklin St. bet. Milton St. and Noble St.,Franklin St. bet. Milton St. and Noble St.,DPR,Playground,0.896,100004272.0,Property,2021-03-12 13:51:03.363,b'\x9f\xba\x91\xda1\xd9\xae\x1eT\x91\xfe\xd7P[...,False
1,B002,B002,B,18,Amersfort Park,Amersfort Park,"E. 38 St., E. 38 St. bet. Ave. I and Ave. J","E. 38 St., E. 38 St. bet. Ave. I and Ave. J",DPR,Neighborhood Park,3.557,100004500.0,Property,2021-03-12 13:51:03.363,b'\x8c\xf0&\xbe 2{;\xa9N\xe06\r~\x901L\xf5\x1c...,False
2,B003,B003,B,19,Bartel-Pritchard Square,Bartel-Pritchard Square,"Prospect Park W., 15 St.","Prospect Park W., 15 St.",DPR,Triangle/Plaza,1.711,100003839.0,Property,2021-03-12 13:51:03.363,b'\xb3m\xd3\x9f\xcb\x01\xfd\xd94\x88\x0c\xeap\...,False
3,B006,B006,B,03,Beattie Square,Beattie Square,"Broadway, Stuyvesant Ave., Vernon Ave.","Broadway, Stuyvesant Ave., Vernon Ave.",DPR,Triangle/Plaza,0.012,100003773.0,Property,2021-03-12 13:51:03.363,"b'o\x98\x94\xf2\xfdH\xd36j\t/|\x0e\\?,\xb3\x87...",False
4,B007,B007,B,11,Bensonhurst Park,Bensonhurst Park,Cropsey Ave. bet. 21 Ave. and Bay Pkwy.,Cropsey Ave. bet. 21 Ave. and Bay Pkwy.,DPR,Recreation Field/Courts,17.500,100004816.0,Property,2021-03-12 13:51:03.363,b'\x06\xc6YC3\xa8\xa6f\x088\xaf(u\x87F\x9a\xe2...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6376,XZ94,XZ94,X,04,GREENSTREET,GREENSTREET,East 153rd St. Bet. Concourse Village We,East 153rd St. Bet. Concourse Village We,DOT,None,0.034,100006143.0,Greenstreet,2021-03-12 13:51:03.363,b'\x89\xc7\xc1\x03\x15r\xa2\xd9\xa7B\xb4\x10\x...,False
6377,XZ95,XZ95,X,04,GREENSTREET,GREENSTREET,Grand Concourse And Tudor Pl.,Grand Concourse And Tudor Pl.,DOT,None,0.002,100007214.0,Greenstreet,2021-03-12 13:51:03.363,b'<\x9a\xec\xf1\xba\x02\x8a\x92G\xdck\x930\xb2...,False
6378,XZ96,XZ96,X,04,GREENSTREET,GREENSTREET,Tudor Pl. And Grand Concourse,Tudor Pl. And Grand Concourse,DOT,None,0.002,100007213.0,Greenstreet,2021-03-12 13:51:03.363,b'\xc34m\xde\x8e\xab\xd2\xc7M\x1f\x93\xaf\xf3\...,False
6379,XZ97,XZ97,X,04,GREENSTREET,GREENSTREET,Mc Clellan St. And Grand Concourse,Mc Clellan St. And Grand Concourse,DOT,None,0.002,100006301.0,Greenstreet,2021-03-12 13:51:03.363,b'Rl\xda\x8a<H\xfa\x1bH\xefk\xc8\xe5\xfa\xec\x...,False


## Perform DML Steps

In [71]:
#Define the common where clause
where = str(r"where objectid like '%3%'")

In [72]:
where

"where objectid like '%3%'"

In [77]:
#Loop through the GIS datasets, drop the _evw and perform the deletes
sql = ['delete from parksgis.dpr.{} {}'.format(g.replace('_evw', ''), where) for g in gis_tables]

In [78]:
gis_con = gis_engine.connect()

In [79]:
sql

["delete from parksgis.dpr.property where objectid like '%3%'",
 "delete from parksgis.dpr.playground where objectid like '%3%'",
 "delete from parksgis.dpr.zone where objectid like '%3%'",
 "delete from parksgis.dpr.unmapped_gisallsites where objectid like '%3%'",
 "delete from parksgis.dpr.schoolyard_to_playground where objectid like '%3%'",
 "delete from parksgis.dpr.greenstreet where objectid like '%3%'",
 "delete from parksgis.dpr.golfcourse where objectid like '%3%'",
 "delete from parksgis.dpr.restrictivedeclarationsite where objectid like '%3%'"]

In [76]:
for q in sql:
    gis_con.execute(q)

ProgrammingError: (pyodbc.ProgrammingError) ('42S02', "[42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid object name 'property'. (208) (SQLExecDirectW)")
[SQL: delete from property where objectid like '%3%']
(Background on this error at: http://sqlalche.me/e/f405)